# 誤差反向傳播反
上一章節說明了神經網路的「學習」。此時，神經網路權重參數的梯度( 精確來說是與參數有關的損失函數梯度 )，是利用數值微分計算出來。數值微分很簡單，執行起來也不難，但運算卻較花時間。因此本章要學習的「誤差反向傳播法」，是能以良好效率計算出權重參數梯度的方法。<br><br>

計算誤差反向傳播法有兩種方法，一是利用「算式」，另一種是用「計算圖( computational graph )」。前者雖簡潔但可能忽略掉本質，迷失於算式中，後者則反之。



## 5.1 計算圖
計算圖中的+節點代表複合函數從內到外拆解( forward )，$\times$節點則代表複合函數由外到內拆解( backward )，而這兩種拆解動作作用於計算圖上時，是以「層級」為單位。


## 5.2 執行單純的層級
層級是用來執行forward()和backward()等共通方法( 介面 )的部分。 forward()是對應正向傳播，backward()則對應到反向傳播。<br><br>
接著要執行乘法層。


In [ ]:
class MulLayer:
    def __init__(self):
        self.x = None
        self.y = None
        
    def forward(self, x, y):
        self.x = x
        self.y = y
        out = x* y
        
        return out
    def backward(self, dout):
        dx = dout * self.y #x與y相反
        dy = dout * self.x
        
        return dx, dy
    

### 解說
在 __init__中，進行實例變數x與y的初始化，用來保持正向傳播時的輸入值。在forward()中，取得x與y等兩個引數。然而，在backward()中，針對上層傳來的微分( dout )，乘上正向傳播的「相反值」在傳遞給下層。


## 5.3 執行活化函數層
接下來我們將計算圖的思考方法套用在神經網路上，這裡把神經網路的「層」當一個類別來處理。首先，要執行活化函數ReLU與Sigmoid層。


### ReLU層

In [ ]:
# ReLU層
class Relu:
    def __init__(self):
        self.mask = None # 實例變數mask
                         # 是由True / False 組成的Numpy陣列
    
    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0
        
        return out
    
    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout
        
        return dx

# 如果正向傳播時的輸入值小於0，反向傳播的值就會變成0
# 因此，在反向傳播中，使用正向傳播的mask，針對上層傳來的dout
# 將mask元素為True的位置設定為0


### Sigmoid層
Sigmoid函數為：<br>
$
y = \frac{1}{1+exp(-x)} \tag{5.1}
$<br>
其中，裡面運算子元素有「$\times$」、「+」、「exp」、「/」節點。<br>



In [ ]:
import numpy as np
class Sigmoid:
    def __init__(self):
        self.out = None
    
    def forward(self, x):
        out = 1 / (1+ np.exp(-x))
        self.out = out
    
        return out
    
    def backward(self, dout):
        dx = dout * (1 - self.out) # 由公式解推導而得到!( p132 ) 
        
        return dx


## 5.4 執行 Attine 
### Affine 層 ( 仿射轉換 )
在神經網路的正向傳播中，使用了矩陣乘積(np.dot())，計算含有權重訊號的總和。


In [1]:
class Affine: #證明在p134~p136
    def __init__(self, W, b):
        self.W = W
        self.b = b
        self.x = None
        self.dW = None
        self.db = None
        
    def forward(self, x):
        self.x = x
        out = np.dot(x, self.W) + self.b
        
        return out
    
    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)
        
        return dx
    

## 5.5 執行 Softmax 層
最後要說明的是softmax函數。Soft函數是將輸入值正規化後再輸出。如下圖所示。<br>
![5.5 softmax](./img/5-5.jpg)<br>
由圖片中可知，神經網路的處理分成「推論」以及「學習」階段，解釋如圖。<br>
接下來要執行的softmax層，這裡要執行的是，包含損失函數的交叉熵誤差( cross entropy error )之「 Softmax-with-Loss 層 」。<br>
![5.5 softmax with loss](./img/5.5.1.PNG)<br>
 
由上圖可知，來自Softmax的層的反向傳播，形成($y_1-t_1, y_2-t_2, y_3-t_3$)這樣的「整齊」結果。($y_1,y_2,y_3$)是Softmax層的輸出，($t_1, t_2, t_3$)則是訓練資料，所以($y_1-t_1, y_2-t_2, y_3-t_3$)是Softmax層的輸出與訓練資料的差分。因為，在神經網路的反向傳播中，這個差分的誤差會傳遞給上一層。對於神經網路的學習而言，是非常重要的性質。<br><br>

神經網路的學習目的是，調整權重參數，讓經網路的輸出(Softmax的輸出)趨近訓練資料。因此神經網路的輸出與訓練吃資料的誤差，必須有效率地傳送給上一層。剛才($y_1-t_1, y_2-t_2, y_3-t_3$)的結果等於Softmax層輸出與訓練資料的差直接表現出目前神經網路的輸出與訓練資料的誤差。<br>

{note}使用MSE當作Softmax函數的損失函數時，反向傳播也會得到($y_1-t_1, y_2-t_2, y_3-t_3$)這種整齊的結果。<br><br>


In [2]:
import numpy as np

def cross_entropy_error(y, t):
    delta = 1e-7
    return -np.sum(t * np.log(y + delta))

def softmax (a):
    c = np.max(a)
    exp_a = np.exp(a-c) #防範溢位!
    sum_exp_a = np.sum(exp_a)
    y = exp_a / sum_exp_a
    
    return y

class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None # 損失函數
        self.y = None    # Softmax的輸出
        self.t = None    # 訓練資料( one-hot vector ) 
    
    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        
    def backward(self, dout = 1):
        batch_size = self.t.shape[0]
        dx = (self.y - self.t) / batch_size #此處以計算圖叫好理解 (看上面的圖)
        
        return dx


## 5.6 執行誤差反向傳播法
組合上一節執行過的各個層級，就能像組合樂高般，建構出神經網路。因此，這裡要一邊組合前面執行過的各層，一邊建構神經網路。<br><br>


### 5.6.1 神經網路的學習總圖
在實際開始執行前，我們先再次確認神經網路的學習總圖。接著再列出神經網路的學習步驟。<br><br>

* 前提<br>
神經網路具有可適應的權重與偏權值，調整權重與偏權值，以適應訓練資料，這種過程稱作「學習」。神經網路的學習可以分成以下4步驟進行。<br><br>

* 步驟一( 小批次 )<br>
從訓練資料中，隨機抽取部分資料。

* 步驟二( 計算梯度 )<br>
計算與各權重參數有關的損失函數梯度。

* 步驟三( 更新參數 )<br>
往梯度方向微量更新權重參數。

* 步驟四( 重複 )<br>
重複以上步驟。<br><br>

與前面說明過的不同的是，步驟二的「計算梯度」。上一章節使用了數值微分來算梯度，數值微分可以輕易執行，但是相對來說，需要花的時間與之相比會較高。


### 5.6.2 執行對應誤差反向傳播法的神經網路
接下來，我們將以TwoLayerNet來執行。首先，整理這個類別的實例變數與方法如下圖。<br>
![5.6誤差反向傳播法的神經網路](./img/5.6.jpg) <br><br>

大部分內容與上一章的「4.5執行學習演算法」共通。與之不同的是，這裡主要使用的是層級。利用層級傳播，就能達到處理辨識結果(predict())、計算梯度(gradient())的目的。


In [3]:
import sys, os
sys.path.append(os.path.abspath('./dl_ex')) #載入父目錄檔案的設定
import numpy as np
from dl_ex.common.layers import *
from dl_ex.common.gradient import numerical_gradient
from collections import OrderedDict

class TwoLayerNet:

    def __init__(self, input_size, hidden_size, output_size, 
                 weight_init_std=0.01):
        # 權重初始化
        self.params = {}
        self.params['W1'] = weight_init_std * \
                            np.random.randn(input_size, hidden_size)
        
        self.params['b1'] = np.zeros(hidden_size)
        
        self.params['W2'] = weight_init_std * \
                            np.random.randn(hidden_size, output_size)
        
        self.params['b2'] = np.zeros(output_size)

        # 產生各層級
        self.layers = OrderedDict()
        self.layers['Affine1'] = \
            Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = \
            Affine(self.params['W2'], self.params['b2'])
        
        self.lastLayer = SoftmaxWithLoss()
            
    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)
    
    # x:輸入資料, t:訓練資料
    def loss(self, x, t):
        y = self.predict(x)
        
        return self.lastLayer.forward(y, t)
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        if t.ndim != 1 : 
            t = np.argmax(t, axis = 1)
        
        accuracy = np.sum( y == t) /  float(x.shape[0])
        return accuracy
    
    # x:輸入資料, t:訓練資料
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)
        
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        
        return grads
    
    def gradient(self, x, t):
        # forward
        self.loss(x, t)
        
        # backward
        dout = 1
        dout = self.lastLayer.backward(dout)
        
        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)
            
        # 設定
        grads = {}
        grads['W1'] = self.layers['Affine1'].dW
        grads['b1'] = self.layers['Affine1'].db
        grads['W2'] = self.layers['Affine2'].dW
        grads['b2'] = self.layers['Affine2'].db
        
        return grads
    

### 解說
這裡有個重要的概念，就是把神經網路的層當作OrderedDict這一點極為重要。OrderedDict是有序字典。因此，神經網路的正向傳播，只要按造順序呼叫出各層的forward()方法，就能處理完神經網路的前向傳播。另外，反向傳播也只要依造相反方向來呼叫各層即可。<br><br>

Affine層以及ReLU層各自在內部已經處理好正向傳播與反向傳播，而在此進行的是，按造正確順序連結各層，並且依序( 或相反順序 )呼叫出各層即可。



### 5.6.3 梯度檢查
以數值微分的方式來檢查誤差梯度反向傳播法，這種方式稱為「梯度檢查 ( gradient check )」。


In [7]:
import sys, os
sys.path.append(os.path.abspath('./dl_ex')) #載入父目錄檔案的設定
import numpy as np
from dl_ex.dataset.mnist import load_mnist
from dl_ex.ch05.two_layer_net import TwoLayerNet

# 載入資料
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size = 784, hidden_size = 50, output_size = 10)

x_batch = x_train[:3]
t_batch = t_train[:3]

grad_numerical = network.numerical_gradient(x_batch, t_batch)
grad_backprop = network.gradient(x_batch, t_batch)

# 計算各權重的絕對誤差平均值
for key in grad_numerical.keys():
    diff = np.average( np.abs(grad_backprop[key] - grad_numerical[key]))
    print(key + ":" + str(diff))
    

W1:5.052772720822389e-10
b1:3.051291303630098e-09
W2:5.828749507746217e-09
b2:1.3928355015319083e-07


### 解說
這裡先算出各權重參數的元素差絕對值，在計算出平均值當作誤差。<br><br>

由結果可知，數值微分與反向傳播誤差法計算而得的梯度誤差非常小。


### 5.6.4 使用誤差反向傳播法學習
最後，我們實際用反像誤差進行神經網路的學習。


In [10]:
import sys, os
sys.path.append(os.pardir)

import numpy as np
from dl_ex.dataset.mnist import load_mnist
from dl_ex.ch05.two_layer_net  import TwoLayerNet

# 載入數據
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

iters_num = 20000
train_size = x_train.shape[0]
batch_size = 1000
learning_rate = 0.05

train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_size / batch_size, 1)

for i in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    # 利用誤差反向傳播法計算梯度
    #grad = network.numerical_gradient(x_batch, t_batch)
    grad = network.gradient(x_batch, t_batch)
    
    # 更新
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]
    
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)
    
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print(train_acc, test_acc)


0.11738333333333334 0.1225
0.5079833333333333 0.5131
0.5680666666666667 0.567
0.6948166666666666 0.699
0.7880166666666667 0.7922
0.8204 0.8262
0.8428833333333333 0.8484
0.8579166666666667 0.8618
0.8691833333333333 0.8747
0.8747666666666667 0.8796
0.88105 0.8857
0.8856 0.8884
0.8881166666666667 0.8916
0.8912666666666667 0.8948
0.8942666666666667 0.8974
0.8968666666666667 0.8993
0.8994 0.9017
0.90105 0.9036
0.9023333333333333 0.905
0.9035833333333333 0.9068
0.9051166666666667 0.9071
0.9064 0.9076
0.9084666666666666 0.9096
0.90975 0.9111
0.91015 0.9126
0.9115833333333333 0.9138
0.9123 0.9129
0.9133333333333333 0.9148
0.9145833333333333 0.9167
0.9154 0.9174
0.9166 0.9179
0.9173 0.9182
0.9177333333333333 0.9196
0.9188166666666666 0.9203
0.9189666666666667 0.921
0.92015 0.9229
0.9214333333333333 0.923
0.9222 0.923
0.9222833333333333 0.9253
0.9231833333333334 0.9253
0.9241 0.9259
0.9252333333333334 0.927
0.9254666666666667 0.9274
0.9262166666666667 0.9283
0.92675 0.9291
0.9279666666666667 0.9

## 5.7 重點整理
本章學習使用了視覺化方式顯示計算過程的計算圖。使用計算圖，說明神經網路執行的誤差反向傳播法，還有以層為單位，在神經網路進行處理。例如，ReLU層、Softmax-with-Loss層、Affine層、Softmax層等。在各層中執行forward與backward等方法，正向或反向傳播資料，可以快速計算出權重參數的梯度。利用「層」進行模組化。可以在神經網路中，隨意組合各層，輕鬆製作出想要的網路。<br><br>

* 使用計算圖，可以用視覺化方式掌握計算過程。
* 計算圖的節點是由局部性計算所構成，其能構成整個計算。
* 計算圖的正向傳播是進行一般計算。利用計算圖的反向傳播，可以計算出各節點的微分。
* 把神經網路的構成元素當作「層」來執行處理，可以快速計算出梯度(  亦即誤差反向傳播法 )。
* 比較數值微分與誤差反向傳播法，可以確認誤差反向傳播法的執行過程有沒有錯誤 ( 梯度檢查 )。
